# tsfresh

First prepare data to be in a format compatibile with tsfresh library.
Two approaches will be tested:
- Extracting features from raw signal (minute interval)
- Extracting features from aggregated signal (hour interval)
Result of this extraction will be a set of features for each patient.

In [1]:
import tsfresh as tsf

import os
from typing import Callable, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp

Q:\ProgramData\Miniconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
Q:\ProgramData\Miniconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
Q:\ProgramData\Miniconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [2]:
# Load data
data_dir = "./depresjon_data"
condition_dir = os.path.join(data_dir, "condition")
control_dir = os.path.join(data_dir, "control")

conditions = [pd.read_csv(os.path.join(condition_dir, filename)) for filename in os.listdir(condition_dir)]
controls = [pd.read_csv(os.path.join(control_dir, filename)) for filename in os.listdir(control_dir)]

### Feature selection on raw signal with minute interval 

In [3]:
# 1st part - raw signal (minute interval)

def get_raw_dataframe():
    f_frames = []

    for c_i, c in enumerate(conditions, start=1):
        c_copy = c.copy()
        c_copy['number'] = f'condition_{c_i}'
        f_frames.append(c_copy)

    for c_i, c in enumerate(controls, start=1):
        c_copy = c.copy()
        c_copy['number'] = f'control_{c_i}'
        f_frames.append(c_copy)

    f_df = pd.concat(f_frames)
    f_df.reset_index(inplace=True, drop=True)
    f_df.drop(inplace=True, columns='date')
    return f_df

In [4]:
raw_df = get_raw_dataframe()
raw_df

,timestamp,activity,number
0,2003-05-07 12:00:00,0,condition_1
1,2003-05-07 12:01:00,143,condition_1
2,2003-05-07 12:02:00,0,condition_1
3,2003-05-07 12:03:00,20,condition_1
4,2003-05-07 12:04:00,166,condition_1
...,...,...,...
1571701,2003-12-01 12:53:00,7,control_32
1571702,2003-12-01 12:54:00,7,control_32
1571703,2003-12-01 12:55:00,5,control_32
1571704,2003-12-01 12:56:00,5,control_32


In [5]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1571706 entries, 0 to 1571705
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   timestamp  1571706 non-null  object
 1   activity   1571706 non-null  int64 
 2   number     1571706 non-null  object
dtypes: int64(1), object(2)
memory usage: 36.0+ MB


In [6]:
raw_df['timestamp']= pd.to_datetime(raw_df['timestamp'])
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1571706 entries, 0 to 1571705
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   timestamp  1571706 non-null  datetime64[ns]
 1   activity   1571706 non-null  int64         
 2   number     1571706 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 36.0+ MB


In [7]:
raw_df_grouped = list(raw_df.groupby('number'))

In [8]:
raw_df_grouped[0][1]

,timestamp,activity,number
0,2003-05-07 12:00:00,0,condition_1
1,2003-05-07 12:01:00,143,condition_1
2,2003-05-07 12:02:00,0,condition_1
3,2003-05-07 12:03:00,20,condition_1
4,2003-05-07 12:04:00,166,condition_1
...,...,...,...
23239,2003-05-23 15:19:00,0,condition_1
23240,2003-05-23 15:20:00,0,condition_1
23241,2003-05-23 15:21:00,0,condition_1
23242,2003-05-23 15:22:00,0,condition_1


In [14]:
from tsfresh.feature_extraction.settings import MinimalFCParameters, EfficientFCParameters
from tsfresh.utilities.dataframe_functions import impute

fc_parameters = EfficientFCParameters()

raw_df_features = []

for name, df in raw_df_grouped:
    df_features = tsf.extract_features(df, 
                                       column_id="number",
                                       column_sort="timestamp",
                                       column_value='activity',
                                       default_fc_parameters=fc_parameters)
    df_features['number'] = name
    raw_df_features.append(df_features)
    
raw_df_features = pd.concat(raw_df_features)
raw_df_features.reset_index(inplace=True, drop=True)

Feature Extraction: 100%|███████████████████████████████████████████████████████████████████████████| 1/1 [00:22<00:00, 22.93s/it]


In [32]:
raw_df_features['number'] = raw_df_features['number'].astype("string")
raw_df_features.sort_values(by='number', inplace=True)
print(raw_df_features.info())
raw_df_features

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 0 to 54
Columns: 782 entries, activity__variance_larger_than_standard_deviation to number
dtypes: float64(781), string(1)
memory usage: 336.4 KB
None


,activity__variance_larger_than_standard_deviation,activity__has_duplicate_max,activity__has_duplicate_min,activity__has_duplicate,activity__sum_values,activity__abs_energy,activity__mean_abs_change,activity__mean_change,activity__mean_second_derivative_central,activity__median,...,activity__permutation_entropy__dimension_6__tau_1,activity__permutation_entropy__dimension_7__tau_1,activity__query_similarity_count__query_None__threshold_0.0,"activity__matrix_profile__feature_""min""__threshold_0.98","activity__matrix_profile__feature_""max""__threshold_0.98","activity__matrix_profile__feature_""mean""__threshold_0.98","activity__matrix_profile__feature_""median""__threshold_0.98","activity__matrix_profile__feature_""25""__threshold_0.98","activity__matrix_profile__feature_""75""__threshold_0.98",number
0,1.0,0.0,1.0,1.0,3415660.0,2.519061e+09,92.156133,0.022932,0.008390,9.0,...,4.790671,5.926574,NaN,10.717293,35.042831,24.557072,24.854357,22.338944,27.377075,condition_1
1,1.0,1.0,1.0,1.0,1517859.0,5.035404e+08,62.871920,0.015272,0.007636,3.0,...,5.036623,6.248507,NaN,2.646326,18.547237,12.380500,12.587729,11.473409,13.519137,condition_10
2,1.0,0.0,1.0,1.0,3338367.0,2.273304e+09,99.031512,-0.011729,0.004239,19.0,...,5.104396,6.332625,NaN,2.872591,18.547237,11.886663,11.989251,10.801740,13.029430,condition_11
3,1.0,0.0,1.0,1.0,5981554.0,5.178708e+09,82.280899,-0.037714,0.005935,9.0,...,4.437789,5.457963,NaN,7.255049,35.584583,27.131746,27.614313,25.032306,30.140960,condition_12
4,1.0,0.0,1.0,1.0,1413779.0,6.448849e+08,40.059197,0.013697,0.000967,0.0,...,3.277779,3.974343,NaN,8.831307,33.585711,24.640861,24.306735,21.811059,26.709031,condition_13
5,1.0,0.0,1.0,1.0,1628308.0,1.076168e+09,56.492434,0.000000,-0.008518,0.0,...,3.635424,4.413400,NaN,2.789047,14.603056,10.305244,10.549469,9.206000,11.654270,condition_14
6,1.0,0.0,1.0,1.0,3521753.0,1.968854e+09,87.974186,0.000000,0.003583,21.0,...,4.816745,5.942217,NaN,2.564762,12.186397,8.741441,9.036110,7.549527,10.094201,condition_15
7,1.0,0.0,1.0,1.0,6379462.0,5.313604e+09,104.934538,-0.005114,0.001286,0.0,...,4.134169,5.085019,NaN,9.138115,40.543257,32.444698,34.470584,31.661410,36.172590,condition_16
8,1.0,0.0,1.0,1.0,5743208.0,4.985132e+09,123.785929,0.000000,-0.000023,50.0,...,5.378909,6.704499,NaN,6.865422,28.142495,18.965327,19.404571,17.446783,20.854436,condition_17
9,1.0,0.0,1.0,1.0,5925033.0,5.917171e+09,150.378381,-0.023660,-0.002946,61.0,...,5.143449,6.384837,NaN,7.652022,35.700874,29.149489,29.668191,27.177986,31.535803,condition_18


In [16]:
raw_df_features.to_csv('pre_impute_efficient.csv')

In [31]:
raw_df_features

,activity__variance_larger_than_standard_deviation,activity__has_duplicate_max,activity__has_duplicate_min,activity__has_duplicate,activity__sum_values,activity__abs_energy,activity__mean_abs_change,activity__mean_change,activity__mean_second_derivative_central,activity__median,...,activity__permutation_entropy__dimension_6__tau_1,activity__permutation_entropy__dimension_7__tau_1,activity__query_similarity_count__query_None__threshold_0.0,"activity__matrix_profile__feature_""min""__threshold_0.98","activity__matrix_profile__feature_""max""__threshold_0.98","activity__matrix_profile__feature_""mean""__threshold_0.98","activity__matrix_profile__feature_""median""__threshold_0.98","activity__matrix_profile__feature_""25""__threshold_0.98","activity__matrix_profile__feature_""75""__threshold_0.98",number
0,1.0,0.0,1.0,1.0,3415660.0,2.519061e+09,92.156133,0.022932,0.008390,9.0,...,4.790671,5.926574,NaN,10.717293,35.042831,24.557072,24.854357,22.338944,27.377075,condition_1
1,1.0,1.0,1.0,1.0,1517859.0,5.035404e+08,62.871920,0.015272,0.007636,3.0,...,5.036623,6.248507,NaN,2.646326,18.547237,12.380500,12.587729,11.473409,13.519137,condition_10
2,1.0,0.0,1.0,1.0,3338367.0,2.273304e+09,99.031512,-0.011729,0.004239,19.0,...,5.104396,6.332625,NaN,2.872591,18.547237,11.886663,11.989251,10.801740,13.029430,condition_11
3,1.0,0.0,1.0,1.0,5981554.0,5.178708e+09,82.280899,-0.037714,0.005935,9.0,...,4.437789,5.457963,NaN,7.255049,35.584583,27.131746,27.614313,25.032306,30.140960,condition_12
4,1.0,0.0,1.0,1.0,1413779.0,6.448849e+08,40.059197,0.013697,0.000967,0.0,...,3.277779,3.974343,NaN,8.831307,33.585711,24.640861,24.306735,21.811059,26.709031,condition_13
5,1.0,0.0,1.0,1.0,1628308.0,1.076168e+09,56.492434,0.000000,-0.008518,0.0,...,3.635424,4.413400,NaN,2.789047,14.603056,10.305244,10.549469,9.206000,11.654270,condition_14
6,1.0,0.0,1.0,1.0,3521753.0,1.968854e+09,87.974186,0.000000,0.003583,21.0,...,4.816745,5.942217,NaN,2.564762,12.186397,8.741441,9.036110,7.549527,10.094201,condition_15
7,1.0,0.0,1.0,1.0,6379462.0,5.313604e+09,104.934538,-0.005114,0.001286,0.0,...,4.134169,5.085019,NaN,9.138115,40.543257,32.444698,34.470584,31.661410,36.172590,condition_16
8,1.0,0.0,1.0,1.0,5743208.0,4.985132e+09,123.785929,0.000000,-0.000023,50.0,...,5.378909,6.704499,NaN,6.865422,28.142495,18.965327,19.404571,17.446783,20.854436,condition_17
9,1.0,0.0,1.0,1.0,5925033.0,5.917171e+09,150.378381,-0.023660,-0.002946,61.0,...,5.143449,6.384837,NaN,7.652022,35.700874,29.149489,29.668191,27.177986,31.535803,condition_18


In [36]:
# Get ids as separate DataFrame
raw_df_features_ids = pd.DataFrame(raw_df_features['number'].copy())
raw_df_features_ids.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 0 to 54
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   number  55 non-null     string
dtypes: string(1)
memory usage: 880.0 bytes


In [39]:
# Drop ids from origin DataFrame before imputing NaNs
raw_df_features.drop(columns='number', inplace=True)
raw_df_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 0 to 54
Columns: 781 entries, activity__variance_larger_than_standard_deviation to activity__matrix_profile__feature_"75"__threshold_0.98
dtypes: float64(781)
memory usage: 336.0 KB


In [40]:
# Replace all NaNs values using impute
impute(raw_df_features)
raw_df_features

Q:\ProgramData\Miniconda3\lib\site-packages\tsfresh\utilities\dataframe_functions.py:171: RuntimeWarning: The columns ['activity__friedrich_coefficients__coeff_0__m_3__r_30'
 'activity__friedrich_coefficients__coeff_1__m_3__r_30'
 'activity__friedrich_coefficients__coeff_2__m_3__r_30'
 'activity__friedrich_coefficients__coeff_3__m_3__r_30'
 'activity__max_langevin_fixed_point__m_3__r_30'
 'activity__query_similarity_count__query_None__threshold_0.0'] did not have any finite values. Filling with zeros.
  warnings.warn("The columns {} did not have any finite values. Filling with zeros.".format(


,activity__variance_larger_than_standard_deviation,activity__has_duplicate_max,activity__has_duplicate_min,activity__has_duplicate,activity__sum_values,activity__abs_energy,activity__mean_abs_change,activity__mean_change,activity__mean_second_derivative_central,activity__median,...,activity__permutation_entropy__dimension_5__tau_1,activity__permutation_entropy__dimension_6__tau_1,activity__permutation_entropy__dimension_7__tau_1,activity__query_similarity_count__query_None__threshold_0.0,"activity__matrix_profile__feature_""min""__threshold_0.98","activity__matrix_profile__feature_""max""__threshold_0.98","activity__matrix_profile__feature_""mean""__threshold_0.98","activity__matrix_profile__feature_""median""__threshold_0.98","activity__matrix_profile__feature_""25""__threshold_0.98","activity__matrix_profile__feature_""75""__threshold_0.98"
0,1.0,0.0,1.0,1.0,3415660.0,2.519061e+09,92.156133,0.022932,0.008390,9.0,...,3.620590,4.790671,5.926574,0.0,10.717293,35.042831,24.557072,24.854357,22.338944,27.377075
1,1.0,1.0,1.0,1.0,1517859.0,5.035404e+08,62.871920,0.015272,0.007636,3.0,...,3.791181,5.036623,6.248507,0.0,2.646326,18.547237,12.380500,12.587729,11.473409,13.519137
2,1.0,0.0,1.0,1.0,3338367.0,2.273304e+09,99.031512,-0.011729,0.004239,19.0,...,3.839281,5.104396,6.332625,0.0,2.872591,18.547237,11.886663,11.989251,10.801740,13.029430
3,1.0,0.0,1.0,1.0,5981554.0,5.178708e+09,82.280899,-0.037714,0.005935,9.0,...,3.402544,4.437789,5.457963,0.0,7.255049,35.584583,27.131746,27.614313,25.032306,30.140960
4,1.0,0.0,1.0,1.0,1413779.0,6.448849e+08,40.059197,0.013697,0.000967,0.0,...,2.532537,3.277779,3.974343,0.0,8.831307,33.585711,24.640861,24.306735,21.811059,26.709031
5,1.0,0.0,1.0,1.0,1628308.0,1.076168e+09,56.492434,0.000000,-0.008518,0.0,...,2.791675,3.635424,4.413400,0.0,2.789047,14.603056,10.305244,10.549469,9.206000,11.654270
6,1.0,0.0,1.0,1.0,3521753.0,1.968854e+09,87.974186,0.000000,0.003583,21.0,...,3.639686,4.816745,5.942217,0.0,2.564762,12.186397,8.741441,9.036110,7.549527,10.094201
7,1.0,0.0,1.0,1.0,6379462.0,5.313604e+09,104.934538,-0.005114,0.001286,0.0,...,3.155291,4.134169,5.085019,0.0,9.138115,40.543257,32.444698,34.470584,31.661410,36.172590
8,1.0,0.0,1.0,1.0,5743208.0,4.985132e+09,123.785929,0.000000,-0.000023,50.0,...,4.027684,5.378909,6.704499,0.0,6.865422,28.142495,18.965327,19.404571,17.446783,20.854436
9,1.0,0.0,1.0,1.0,5925033.0,5.917171e+09,150.378381,-0.023660,-0.002946,61.0,...,3.871373,5.143449,6.384837,0.0,7.652022,35.700874,29.149489,29.668191,27.177986,31.535803


In [41]:
raw_df_features.to_csv('post_impute_efficient.csv')
raw_df_features_ids.to_csv('number_ids.csv')

In [58]:
raw_df_features_ids.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 0 to 54
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   number  55 non-null     string
dtypes: string(1)
memory usage: 880.0 bytes


In [60]:
raw_df_features_ids['ill'] = np.where(raw_df_features_ids['number'].str.startswith('control'), 0, 1)
raw_df_features_ids

,number,ill
0,condition_1,1
1,condition_10,1
2,condition_11,1
3,condition_12,1
4,condition_13,1
5,condition_14,1
6,condition_15,1
7,condition_16,1
8,condition_17,1
9,condition_18,1


In [61]:
raw_df_features_ids.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 0 to 54
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   number  55 non-null     string
 1   ill     55 non-null     int32 
dtypes: int32(1), string(1)
memory usage: 1.1 KB


In [62]:
raw_df_features_relevant = tsf.select_features(raw_df_features, raw_df_features_ids['ill'])
raw_df_features_relevant

,activity__lempel_ziv_complexity__bins_2,activity__lempel_ziv_complexity__bins_3,activity__count_above_mean,"activity__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""var""","activity__linear_trend__attr_""intercept""","activity__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","activity__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""","activity__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""mean""","activity__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""mean""","activity__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""max""",activity__energy_ratio_by_chunks__num_segments_10__segment_focus_1,"activity__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""max""","activity__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""","activity__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min"""
0,0.061521,0.064017,6328.0,37334.846538,171.032719,328.414831,171.028136,171.024387,171.018320,434.740982,0.059432,794.975200,29838.955311,58.475659
1,0.081604,0.086007,6148.0,14324.740705,75.974977,187.277463,75.946643,75.941083,75.978374,270.287119,0.091301,527.303147,11813.637490,12.630048
2,0.066412,0.066412,5761.0,59661.371852,157.247675,313.718814,157.306488,157.526885,157.381583,434.807369,0.033183,916.733687,39074.423243,54.105278
3,0.007193,0.009402,9271.0,84905.970772,273.996475,491.536353,273.990744,273.864968,273.972781,636.305201,0.096503,1089.960926,53657.078556,120.185709
4,0.053933,0.055093,4850.0,28300.660399,94.557370,200.624640,94.502275,94.487249,94.532415,282.827388,0.119696,640.358730,16439.891341,27.472213
5,0.073315,0.073754,3842.0,22520.259416,65.651816,132.332307,65.681956,65.719695,65.664485,192.314745,0.057323,481.696030,13029.474278,24.320273
6,0.058010,0.059296,7007.0,28484.454677,159.476385,265.614673,159.506574,159.901615,159.556657,348.970310,0.075507,640.954653,17450.851642,82.825897
7,0.048499,0.048849,9322.0,95654.109063,308.518550,533.242040,308.505643,308.234005,308.515810,688.772268,0.064114,1254.319543,59680.279305,143.768922
8,0.032474,0.035846,6872.0,60848.463481,205.044401,338.356997,205.113531,205.255393,205.110299,432.646079,0.047123,815.297230,29676.838134,106.262105
9,0.056226,0.056829,6576.0,54088.927202,196.748169,388.789398,196.893988,198.365849,196.909091,512.191208,0.018230,956.392781,33081.930287,71.821251


In [63]:
raw_df_features_relevant.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 0 to 54
Data columns (total 14 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   activity__lempel_ziv_complexity__bins_2                                   55 non-null     float64
 1   activity__lempel_ziv_complexity__bins_3                                   55 non-null     float64
 2   activity__count_above_mean                                                55 non-null     float64
 3   activity__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"var"   55 non-null     float64
 4   activity__linear_trend__attr_"intercept"                                  55 non-null     float64
 5   activity__agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"max"    55 non-null     float64
 6   activity__agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"me

In [64]:
raw_df_features_relevant.to_csv('raw_df_features_relevant')

### Feature selection on aggregated signal with hour interval 

In [70]:
def group_by_hour(df: pd.DataFrame) -> pd.DataFrame:
    """
    df: DataFrame with columns "datetime" and "activity"
    """
    grouped = df.groupby([pd.Grouper(key="number"), pd.Grouper(key="timestamp", freq="H")]).mean()
    grouped.reset_index(inplace=True)
    return grouped

In [71]:
raw_df_hourly = group_by_hour(raw_df)
raw_df_hourly

,number,timestamp,activity
0,condition_1,2003-05-07 12:00:00,346.550000
1,condition_1,2003-05-07 13:00:00,284.566667
2,condition_1,2003-05-07 14:00:00,279.183333
3,condition_1,2003-05-07 15:00:00,218.783333
4,condition_1,2003-05-07 16:00:00,238.550000
...,...,...,...
26225,control_9,2003-12-29 06:00:00,0.000000
26226,control_9,2003-12-29 07:00:00,0.000000
26227,control_9,2003-12-29 08:00:00,30.100000
26228,control_9,2003-12-29 09:00:00,25.850000


In [76]:
raw_df_hourly_features = tsf.extract_features(raw_df_hourly,
                                              column_id='number',
                                              column_sort='timestamp',
                                              column_value='activity',
                                              default_fc_parameters=fc_parameters)
raw_df_hourly_features.reset_index(inplace=True)
raw_df_hourly_features.info()

Feature Extraction: 100%|█████████████████████████████████████████████████████████████████████████| 28/28 [00:06<00:00,  4.42it/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Columns: 782 entries, index to activity__matrix_profile__feature_"75"__threshold_0.98
dtypes: float64(781), object(1)
memory usage: 336.1+ KB


In [80]:
raw_df_hourly_features.rename(columns={'index' : 'number'}, inplace=True)
raw_df_hourly_features['number'] = raw_df_hourly_features['number'].astype("string")
raw_df_hourly_features

,number,activity__variance_larger_than_standard_deviation,activity__has_duplicate_max,activity__has_duplicate_min,activity__has_duplicate,activity__sum_values,activity__abs_energy,activity__mean_abs_change,activity__mean_change,activity__mean_second_derivative_central,...,activity__permutation_entropy__dimension_5__tau_1,activity__permutation_entropy__dimension_6__tau_1,activity__permutation_entropy__dimension_7__tau_1,activity__query_similarity_count__query_None__threshold_0.0,"activity__matrix_profile__feature_""min""__threshold_0.98","activity__matrix_profile__feature_""max""__threshold_0.98","activity__matrix_profile__feature_""mean""__threshold_0.98","activity__matrix_profile__feature_""median""__threshold_0.98","activity__matrix_profile__feature_""25""__threshold_0.98","activity__matrix_profile__feature_""75""__threshold_0.98"
0,condition_1,1.0,0.0,1.0,1.0,56975.366667,2.362613e+07,80.216408,-0.690052,0.168502,...,4.284453,5.176008,5.608785,NaN,1.005856,4.166246,2.568924,2.579452,2.285965,2.758028
1,condition_10,1.0,0.0,1.0,1.0,25311.561765,3.075595e+06,43.914574,0.054527,-0.436415,...,4.486936,5.475311,5.793644,NaN,0.847707,3.108518,1.992460,2.003186,1.747461,2.252696
2,condition_11,1.0,0.0,1.0,1.0,55714.483333,1.888407e+07,90.401036,-0.423917,0.127927,...,4.386740,5.292649,5.643388,NaN,1.324239,4.017961,2.506381,2.437698,2.127063,2.806442
3,condition_12,1.0,0.0,0.0,1.0,99695.802899,5.372026e+07,92.042777,-0.533972,0.168356,...,4.283828,5.359698,5.890302,NaN,0.901890,4.198842,2.276803,2.180840,1.919083,2.550376
4,condition_13,1.0,0.0,1.0,1.0,23654.646078,3.689933e+06,45.145157,-0.275570,0.050228,...,4.106355,4.866261,5.106962,NaN,8.767380,10.865761,9.825534,9.874470,9.490995,10.148890
5,condition_14,1.0,0.0,0.0,1.0,27160.306667,6.516002e+06,72.088226,-0.109474,-0.005733,...,4.546966,5.428273,5.700243,NaN,1.241496,3.726190,2.631365,2.656337,2.283431,2.937811
6,condition_15,1.0,0.0,1.0,1.0,58783.831982,1.693174e+07,91.613640,-0.295366,-0.181885,...,4.576314,5.551079,5.824859,NaN,1.080196,3.662818,2.756374,2.800171,2.468637,3.074342
7,condition_16,1.0,0.0,1.0,1.0,106367.305556,5.306206e+07,109.095812,-0.540829,0.284438,...,4.395799,5.430235,5.884249,NaN,2.003132,5.189370,3.671526,3.682174,3.303828,4.083913
8,condition_17,1.0,0.0,0.0,1.0,95725.937500,5.202901e+07,140.008576,0.057095,0.025702,...,4.430366,5.353292,5.710977,NaN,0.826654,3.830329,1.941721,1.880501,1.605593,2.245143
9,condition_18,1.0,0.0,1.0,1.0,99252.150000,6.405131e+07,133.687130,-1.416296,0.391527,...,4.291354,5.209021,5.580560,NaN,0.887149,3.578342,2.165096,2.133523,1.785377,2.559410


In [81]:
raw_df_hourly_features_ids = pd.DataFrame(raw_df_hourly_features['number'].copy())
raw_df_hourly_features_ids['ill'] = np.where(raw_df_hourly_features_ids['number'].str.startswith('control'), 0, 1)

raw_df_hourly_features.drop(columns='number', inplace=True)
raw_df_hourly_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Columns: 781 entries, activity__variance_larger_than_standard_deviation to activity__matrix_profile__feature_"75"__threshold_0.98
dtypes: float64(781)
memory usage: 335.7 KB


In [87]:
impute(raw_df_hourly_features)
raw_df_hourly_features

,activity__variance_larger_than_standard_deviation,activity__has_duplicate_max,activity__has_duplicate_min,activity__has_duplicate,activity__sum_values,activity__abs_energy,activity__mean_abs_change,activity__mean_change,activity__mean_second_derivative_central,activity__median,...,activity__permutation_entropy__dimension_5__tau_1,activity__permutation_entropy__dimension_6__tau_1,activity__permutation_entropy__dimension_7__tau_1,activity__query_similarity_count__query_None__threshold_0.0,"activity__matrix_profile__feature_""min""__threshold_0.98","activity__matrix_profile__feature_""max""__threshold_0.98","activity__matrix_profile__feature_""mean""__threshold_0.98","activity__matrix_profile__feature_""median""__threshold_0.98","activity__matrix_profile__feature_""25""__threshold_0.98","activity__matrix_profile__feature_""75""__threshold_0.98"
0,1.0,0.0,1.0,1.0,56975.366667,2.362613e+07,80.216408,-0.690052,0.168502,79.100000,...,4.284453,5.176008,5.608785,0.0,1.005856,4.166246,2.568924,2.579452,2.285965,2.758028
1,1.0,0.0,1.0,1.0,25311.561765,3.075595e+06,43.914574,0.054527,-0.436415,64.666667,...,4.486936,5.475311,5.793644,0.0,0.847707,3.108518,1.992460,2.003186,1.747461,2.252696
2,1.0,0.0,1.0,1.0,55714.483333,1.888407e+07,90.401036,-0.423917,0.127927,116.716667,...,4.386740,5.292649,5.643388,0.0,1.324239,4.017961,2.506381,2.437698,2.127063,2.806442
3,1.0,0.0,0.0,1.0,99695.802899,5.372026e+07,92.042777,-0.533972,0.168356,22.216667,...,4.283828,5.359698,5.890302,0.0,0.901890,4.198842,2.276803,2.180840,1.919083,2.550376
4,1.0,0.0,1.0,1.0,23654.646078,3.689933e+06,45.145157,-0.275570,0.050228,16.875000,...,4.106355,4.866261,5.106962,0.0,8.767380,10.865761,9.825534,9.874470,9.490995,10.148890
5,1.0,0.0,0.0,1.0,27160.306667,6.516002e+06,72.088226,-0.109474,-0.005733,26.400000,...,4.546966,5.428273,5.700243,0.0,1.241496,3.726190,2.631365,2.656337,2.283431,2.937811
6,1.0,0.0,1.0,1.0,58783.831982,1.693174e+07,91.613640,-0.295366,-0.181885,144.925000,...,4.576314,5.551079,5.824859,0.0,1.080196,3.662818,2.756374,2.800171,2.468637,3.074342
7,1.0,0.0,1.0,1.0,106367.305556,5.306206e+07,109.095812,-0.540829,0.284438,48.783333,...,4.395799,5.430235,5.884249,0.0,2.003132,5.189370,3.671526,3.682174,3.303828,4.083913
8,1.0,0.0,0.0,1.0,95725.937500,5.202901e+07,140.008576,0.057095,0.025702,189.783333,...,4.430366,5.353292,5.710977,0.0,0.826654,3.830329,1.941721,1.880501,1.605593,2.245143
9,1.0,0.0,1.0,1.0,99252.150000,6.405131e+07,133.687130,-1.416296,0.391527,182.466667,...,4.291354,5.209021,5.580560,0.0,0.887149,3.578342,2.165096,2.133523,1.785377,2.559410


In [109]:
raw_df_hourly_features_relevant = tsf.select_features(raw_df_hourly_features, 
                                                      raw_df_hourly_features_ids['ill'],
                                                      fdr_level=0.1)
raw_df_hourly_features_relevant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 24 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   activity__count_above_mean                                                55 non-null     float64
 1   activity__benford_correlation                                             55 non-null     float64
 2   activity__agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"max"    55 non-null     float64
 3   activity__linear_trend__attr_"intercept"                                  55 non-null     float64
 4   activity__agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"mean"   55 non-null     float64
 5   activity__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"mean"  55 non-null     float64
 6   activity__energy_ratio_by_chunks__num_segments_10__segment_focus_1  

In [111]:
raw_df_hourly_features_relevant

,activity__count_above_mean,activity__benford_correlation,"activity__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","activity__linear_trend__attr_""intercept""","activity__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""","activity__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""mean""",activity__energy_ratio_by_chunks__num_segments_10__segment_focus_1,activity__absolute_sum_of_changes,"activity__fft_aggregated__aggtype_""variance""",activity__number_cwt_peaks__n_1,...,"activity__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""var""","activity__fft_coefficient__attr_""abs""__coeff_36",activity__partial_autocorrelation__lag_8,"activity__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""",activity__length,activity__range_count__max_1000000000000.0__min_0,"activity__fft_coefficient__attr_""real""__coeff_0","activity__fft_coefficient__attr_""abs""__coeff_0",activity__cid_ce__normalize_False,activity__sum_values
0,147.0,0.975991,280.081575,171.121853,171.784945,171.723219,0.053533,31043.750000,2977.424946,32.0,...,5100.792707,3644.251420,-0.110850,66.114968,388.0,388.0,56975.366667,56975.366667,3114.564449,56975.366667
1,160.0,0.925692,122.602962,75.733197,75.636065,76.284400,0.075935,15633.588235,2796.834724,34.0,...,877.792000,602.119745,-0.031404,31.879690,357.0,357.0,25311.561765,25311.561765,1231.248039,25311.561765
2,141.0,0.987196,272.899902,157.759723,157.821311,161.595783,0.013731,32001.966667,2988.078291,35.0,...,8291.698959,3159.032492,-0.036299,64.774824,355.0,355.0,55714.483333,55714.483333,2616.965391,55714.483333
3,202.0,0.669186,535.282245,273.830009,272.953971,272.022741,0.087255,59643.719565,9662.355619,55.0,...,38903.698281,713.016317,-0.114642,101.979978,649.0,649.0,99695.802899,99695.802899,4840.724251,99695.802899
4,157.0,0.969500,208.274430,94.757596,94.403790,93.994927,0.115711,19457.562745,4444.277164,34.0,...,5232.617258,3818.624427,-0.043467,25.557955,432.0,432.0,23654.646078,23654.646078,1648.511330,23654.646078
5,115.0,0.981013,137.783002,65.924527,66.914510,68.943129,0.038160,24654.173333,2730.703814,32.0,...,2647.885000,1003.610253,-0.036609,16.762712,343.0,343.0,27160.306667,27160.306667,2343.695885,27160.306667
6,163.0,0.941750,274.855967,159.392122,159.939375,162.320274,0.079702,33255.751351,2913.994837,37.0,...,7409.009761,449.068787,-0.133626,66.566745,364.0,364.0,58783.831982,58783.831982,2400.237874,58783.831982
7,212.0,0.859032,563.827456,308.260544,307.440130,307.596959,0.059826,57166.205556,6377.263979,46.0,...,29901.652110,4207.219300,-0.031804,128.295490,525.0,525.0,106367.305556,106367.305556,4285.101075,106367.305556
8,150.0,0.941621,409.697521,205.477189,212.452912,219.995708,0.033065,50403.087500,3323.500271,38.0,...,21270.213016,2616.270605,-0.167104,93.733798,361.0,361.0,95725.937500,95725.937500,4218.955172,95725.937500
9,124.0,0.985699,364.703517,201.388226,209.665830,219.001963,0.011563,48127.366667,2649.313943,34.0,...,8130.759140,1859.038209,-0.124941,102.670104,361.0,361.0,99252.150000,99252.150000,4106.401107,99252.150000


In [112]:
raw_df_hourly_features_relevant.to_csv('raw_df_hourly_features_relevant.csv')